**Pasos previos**

In [2]:
#Librerias necesarias
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import seaborn as sns
import datetime

from scipy import spatial
import pandas as pd

In [3]:
#Esta celda es para cargar los archivos de prueba de la función. NO SE DEBE COPIAR
filesNames = ['2019_11_Q1','2019_11_Q2','2020_03_Q1',"2020_03_Q2",'2020_04_Q1','2020_04_Q2','2020_05_Q1']
pas=pd.read_csv('2019_11_Q1.csv',header=0,encoding='UTF-8')
#dfs list that contain  the df of each file
#dfs=[df,df,df,df,df,df,df]

In [4]:
vehiculos = pd.read_csv('vehiculos.csv',header=0,engine='python')
vehiculos.head()
data=pas.merge(vehiculos, on='IDVEHICULO', how='left')


**funciones de limpieza**

In [5]:
#GENERAR COLUMNAS DE TRABAJO

#Cambio del formato de las fechas
data['FECHAREGISTRO'] = pd.to_datetime(data['FECHAREGISTRO'], format='%d/%m/%Y %H:%M:%S')

#Columna que representa el dia de la semana. Sirve para diferenciar entre dia habil y fin de semana
data["dia"]=data['FECHAREGISTRO'].dt.dayofweek

#Función para asignar la franja del día
def franja(i):
    pico_mañana_min = 5
    pico_mañana_max = 7
    pico_tarde_min = 14
    pico_tarde_max = 19
    
    if pico_mañana_min<i.hour<pico_mañana_max:
        return "1_peak"
    elif pico_tarde_min<i.hour<pico_tarde_max:
        return "2_peak"
    else:
        return "Off_peak"

#Columna con la franja del dia
data["franja"]=data['FECHAREGISTRO'].apply(franja)

In [6]:
# Esta funcion sirve para eliminar los viajes que tienen menos de 5 registros
# Para evitar un uso de memoria mayor, todas las limpiezas deberian hacer sobre el mismo objeto usando el drop. 
#hace falta ajustar eso
def delta_secrec(data,minval=5):
    aggreg={'IDVEHICULO':"count"}
    filt_reg=data.groupby("SECUENCIARECORRIDO").agg(aggreg)
    filt_reg.rename(columns={'IDVEHICULO':"n"},inplace=True)
    filt_reg.reset_index(inplace=True)
    filt_reg.columns
    rec_fil=list(filt_reg[filt_reg["n"]<minval]["SECUENCIARECORRIDO"].unique())
    index_to_drop=list(data[data["SECUENCIARECORRIDO"].isin(rec_fil)].index[0])
    data.drop(index=index_to_drop, axis=0, inplace=True)

In [7]:
def limit_cap(df,var_list=['SUBENDELANTERA','SUBENTRASERA', 'BAJANDELANTERA', 'BAJANTRASERA'],maxVal=["CAPPASAJEROS"]):
    for var in var_list:
        df[var].where(df[var]<df[maxVal], df[maxVal],inplace=True)  

In [8]:
#Función para calcular la carga en el data frame de entrada
def carga(df):
    df.sort_values(by=["SECUENCIARECORRIDO","FECHAREGISTRO"],inplace=True,ascending=True)
    df['FECHAREGISTRO_d'] = df['FECHAREGISTRO'].dt.day
    df['FECHAREGISTRO_m'] = df['FECHAREGISTRO'].dt.month
    df['delta_q'] = df['SUBENDELANTERA']+df['SUBENTRASERA']-df['BAJANDELANTERA']-df['BAJANTRASERA']
    df["carga"]=df.groupby(["SECUENCIARECORRIDO",'FECHAREGISTRO_d'])["delta_q"].cumsum()

In [9]:
def filtrar_neg(df):
    return data[data[carga]>=0]

In [10]:
#Función de Jose para normalizar las ubicaciones de acuerdo a los archivos kml

#Define new coordinates to the dataframes having into account the KML coordinates
def new_coordinates(dataframe):
    
    #Iterating for every route in the dataframe
    for route in dataframe['CODIGORUTA'].unique():
        
        #Selecting the data of interest
        data = dataframe[dataframe['CODIGORUTA']== route]
        data_tuple = list(zip(data['LATITUD'],data['LONGITUD']))
        
        #reading CSV for geographical information from KML files. If KML file is not found, the new coordinates are 
        #the same as the old coordinates
        try:
            route_data = pd.read_csv(('./Data/Kml ruta AMVA/csvs/{}.csv').format(route))
        except:
            dataframe.loc[dataframe['CODIGORUTA']==route,'nuevas coordenadas'] = pd.Series(data_tuple, index = dataframe.loc[dataframe['CODIGORUTA'] == route].index)
            print(route)
            continue

        #Giving specified format to route data according to the spatial.KDTree function
        route_data['coordinates'] = list(zip(route_data['latitude'],route_data['longitude']))
        route_coordinates = route_data['coordinates']
        route_coordinates_list = route_coordinates.tolist()
        route_coord_df = pd.DataFrame(route_coordinates)

        #Aplying the function. It queries the route coordinate that is closest to each tuple in data tuple
        tree = spatial.KDTree(route_coordinates_list)
        coord_index = tree.query([data_tuple])[1][0]
        
        #Generating a new coordinates list that is going to be added to the dataframe
        new_coordinates_list = []
        for i in coord_index:
            new_coordinates_list.append(route_coord_df[route_coord_df.index==i]['coordinates'].iloc[0])
        
        #Adding the new coordinates list to the dataframe
        dataframe.loc[dataframe['CODIGORUTA'] == route,'nuevas coordenadas'] = pd.Series(new_coordinates_list, index = dataframe.loc[dataframe['CODIGORUTA'] == route].index)

**A partir de aqui se ejecutan las funciones**

In [11]:
data1=delta_secrec(data,minval=5)

MemoryError: Unable to allocate 358. MiB for an array with shape (8, 5868296) and data type int64

In [ ]:
limit_cap(data1,var_list=['SUBENDELANTERA','SUBENTRASERA', 'BAJANDELANTERA', 'BAJANTRASERA'],maxVal=["CAPPASAJEROS"])

In [ ]:
carga(data1)

In [ ]:
data_ok=filtrar_neg(data1)